# Imports y Configuración Inicial

In [7]:
# Imports y configuración
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Paths
ROOT = Path.cwd()
DATA_DIR = ROOT / "CMAPSSData" 
RAW_DIR = Path(r"../CMAPSSData/raw") 
EDA_OUT = ROOT / "eda_outputs"
EDA_OUT.mkdir(exist_ok=True)

# Visual defaults
plt.rcParams["figure.figsize"] = (12, 6)
sns.set_context("talk")

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Funciones Utilitarias

In [8]:
# Funciones reutilizables solicitadas
def read_cmapss(path):
    """Leer archivo C-MAPSS (espacios múltiples) y asignar nombres de columna."""
    col_names = ["unit", "cycle", "op1", "op2", "op3"] + [f"s{i}" for i in range(1, 22)]
    df = pd.read_csv(path, sep=r"\s+", header=None, names=col_names)
    return df

def compute_rul(df):
    """Calcular RUL por unit para dataset de entrenamiento.
       Devuelve df con nueva columna 'RUL' y rul_per_unit dataframe.
    """
    max_cycle = df.groupby("unit")["cycle"].transform("max")
    df = df.copy()
    df["RUL"] = max_cycle - df["cycle"]
    rul_per_unit = df[["unit", "cycle", "RUL"]].copy()
    return df, rul_per_unit

def summary_stats(df):
    """Resumen con percentiles 25,50,75,99 y missing pct por columna."""
    stats = df.describe(percentiles=[.25, .5, .75, .99]).T
    stats = stats.rename(columns={"50%": "50%", "99%": "99%"})
    stats["missing_pct"] = df.isna().mean() * 100
    cols_keep = ["count", "mean", "std", "min", "25%", "50%", "75%", "99%", "max", "missing_pct"]
    return stats[cols_keep].reset_index().rename(columns={"index": "column"})

def per_unit_stats(df):
    """Estadísticas por unidad: cycles_count, cycle_length, first/last val per sensor (flatten)."""
    units = []
    sensors = [c for c in df.columns if c.startswith("s")]
    grouped = df.groupby("unit")
    for u, g in grouped:
        row = {"unit": u, "cycles_count": g["cycle"].nunique(), "cycle_length": g["cycle"].max() - g["cycle"].min() + 1}
        # primeros y últimos valores por sensor (store as semicolon-separated)
        first_vals = {f"first_{s}": g.iloc[0][s] for s in sensors}
        last_vals = {f"last_{s}": g.iloc[-1][s] for s in sensors}
        row.update(first_vals)
        row.update(last_vals)
        units.append(row)
    return pd.DataFrame(units)

def select_representative_units(df, n_random=3):
    """Seleccionar 6 unidades: min length, median length, max length, y 3 aleatorias distintas."""
    lengths = df.groupby("unit")["cycle"].nunique().reset_index(name="len")
    min_u = int(lengths.sort_values("len").iloc[0]["unit"])
    max_u = int(lengths.sort_values("len").iloc[-1]["unit"])
    med_idx = int(len(lengths) // 2)
    med_u = int(lengths.sort_values("len").iloc[med_idx]["unit"])
    # escoger 3 aleatorias distintas de las anteriores
    pool = set(lengths["unit"].unique()) - {min_u, med_u, max_u}
    rnd = list(np.random.choice(list(pool), size=n_random, replace=False))
    selected = [min_u, med_u, max_u] + rnd
    return selected

def rolling_features_by_unit(df, sensors, windows=(5,10,20)):
    """Ejemplo: calcular rolling mean/std y slope (OLS) en ventanas por unidad."""
    out = df.copy()
    for w in windows:
        for s in sensors:
            out[f"{s}_rm_{w}"] = out.groupby("unit")[s].transform(lambda x: x.rolling(window=w, min_periods=1).mean())
            out[f"{s}_rstd_{w}"] = out.groupby("unit")[s].transform(lambda x: x.rolling(window=w, min_periods=1).std().fillna(0))
            # slope via simple linear regression on rolling window
            def slope(x):
                if len(x) < 2:
                    return 0.0
                idx = np.arange(len(x))
                A = np.vstack([idx, np.ones(len(idx))]).T
                m, _ = np.linalg.lstsq(A, x, rcond=None)[0]
                return m
            out[f"{s}_slope_{w}"] = out.groupby("unit")[s].transform(lambda x: x.rolling(window=w, min_periods=2).apply(slope, raw=True).fillna(0))
    return out

def save_fig(fig, filepath):
    """Guardar figura con tight layout y resolución."""
    fig.tight_layout()
    fig.savefig(filepath, dpi=150)
    plt.close(fig)

# Carga de Datos

In [9]:
import pandas as pd
col_names = ["unit","cycle","op1","op2","op3"] + [f"s{i}" for i in range(1,22)]
train_path = RAW_DIR / "train_FD001.txt"
test_path  = RAW_DIR / "test_FD001.txt"
rul_path   = RAW_DIR / "RUL_FD001.txt"

for p in (train_path, test_path, rul_path):
    if not p.exists():
        raise FileNotFoundError(f"No existe: {p}")

train = pd.read_csv(train_path, sep=r"\s+", header=None, names=col_names, engine="python")
test  = pd.read_csv(test_path,  sep=r"\s+", header=None, names=col_names, engine="python")
rul_truth = pd.read_csv(rul_path, sep=r"\s+", header=None, names=["RUL_true"], engine="python")

print("Lectura OK:", train.shape, test.shape, rul_truth.shape)
display(train.head(3))

Lectura OK: (20631, 26) (13096, 26) (100, 1)


,unit,cycle,op1,op2,op3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442


# Resumen general y summary_stats.csv

In [11]:
summary = summary_stats(train)
summary.to_csv(EDA_OUT / "summary_stats.csv", index=False)

# Guardar una imagen tabular legible (simple plot)
fig, ax = plt.subplots(figsize=(14, 10))
sns.heatmap(pd.DataFrame(summary.set_index("column")["mean"]), annot=True, fmt=".2f", cmap="vlag", cbar=False, ax=ax)
ax.set_title("Resumen (media) por columna — vista simplificada")
save_fig(fig, EDA_OUT / "summary_stats_table.png")

# Chequeos de Calidad

In [13]:
qc = {}
qc["n_rows_train"] = len(train)
qc["n_units_train"] = train["unit"].nunique()
qc["n_rows_test"] = len(test)
qc["n_units_test"] = test["unit"].nunique()
qc["na_counts"] = train.isna().sum().to_dict()
qc["cols_constant"] = [c for c in train.columns if train[c].nunique() <= 1]
qc["duplicates"] = train.duplicated().sum()
# monotonicity por unit
mono_issues = []
for u, g in train.groupby("unit"):
    if not g["cycle"].is_monotonic_increasing:
        mono_issues.append(u)
qc["monotonicity_issues_units"] = mono_issues

# imprimir resumen de QC
print("QC summary:")
for k, v in qc.items():
    if k == "na_counts":
        print("na_counts (sample):", {k: v[k] for k in list(v)[:5]})
    else:
        print(k, ":", (v if not isinstance(v, list) else f"{len(v)} units"))

QC summary:
n_rows_train : 20631
n_units_train : 100
n_rows_test : 13096
n_units_test : 100
na_counts (sample): {'unit': 0, 'cycle': 0, 'op1': 0, 'op2': 0, 'op3': 0}
cols_constant : 7 units
duplicates : 0
monotonicity_issues_units : 0 units


# Cálculo de RUL y guardado rul_per_unit.csv 

In [15]:
train_rul, rul_per_unit = compute_rul(train)
rul_per_unit.to_csv(EDA_OUT / "rul_per_unit.csv", index=False)

# añadir RUL al dataframe train en memoria
train = train_rul

# Count units vs cycles plot

In [ ]:
counts = train.groupby("unit")["cycle"].nunique().reset_index(name="n_cycles")
fig, ax = plt.subplots(figsize=(14,6))
sns.barplot(x="unit", y="n_cycles", data=counts, palette="viridis", ax=ax)
ax.set_title("Cycles por unit (FD001)")
ax.set_xlabel("Unit")
ax.set_ylabel("Number of cycles")
ax.set_xticks(ax.get_xticks()[::5]) 
save_fig(fig, EDA_OUT / "count_units_cycles.png")

# Selección de 6 unidades representativas

In [16]:
selected_units = select_representative_units(train, n_random=3)
print("6 unidades seleccionadas:", selected_units)

6 unidades seleccionadas: [39, 79, 69, np.int64(64), np.int64(42), np.int64(97)]


# Trajectory samples plot (6 unidades, sensors s2,s3,s7,s15)

In [ ]:
sensors_plot = ["s2", "s3", "s7", "s15"]
fig, axes = plt.subplots(6, len(sensors_plot), figsize=(16, 18), sharex=False)
for i, u in enumerate(selected_units):
    sub = train[train["unit"]==u]
    for j, s in enumerate(sensors_plot):
        ax = axes[i, j]
        ax.plot(sub["cycle"], sub[s], label=f"unit {u}")
        ax.set_title(f"unit {u} - {s}")
        ax.set_xlabel("cycle")
        ax.set_ylabel(s)
        ax.grid(True)
save_fig(fig, EDA_OUT / "trajectory_samples.png")

# Operative conditions (histograms y hexbin)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(18,5))
for i, op in enumerate(["op1","op2","op3"]):
    sns.histplot(train[op], ax=ax[i], kde=False, bins=30)
    ax[i].set_title(f"Distribución {op}")
save_fig(fig, EDA_OUT / "op_conditions_hist.png")

# hexbin sensor vs op (ejemplo s7 vs op1)
fig, ax = plt.subplots(figsize=(10,8))
hb = ax.hexbin(train["op1"], train["s7"], gridsize=80, cmap="inferno")
ax.set_xlabel("op1")
ax.set_ylabel("s7")
ax.set_title("Hexbin s7 vs op1")
fig.colorbar(hb, ax=ax)
save_fig(fig, EDA_OUT / "sensor_vs_op_hexbin.png")

# Correlación (Pearson y Spearman) y heatmap

In [ ]:
sensors = [c for c in train.columns if c.startswith("s")]
df_sensors = train[sensors].copy()

pearson = df_sensors.corr(method="pearson")
spearman = df_sensors.corr(method="spearman")

# heatmap Pearson
fig, ax = plt.subplots(figsize=(14,12))
sns.heatmap(pearson, cmap="coolwarm", center=0, annot=False, ax=ax)
ax.set_title("Pearson correlation — sensors")
save_fig(fig, EDA_OUT / "correlation_heatmap.png")

pearson.to_csv(EDA_OUT / "pearson_sensors.csv", index=True)
spearman.to_csv(EDA_OUT / "spearman_sensors.csv", index=True)

# PCA (normalizar sensores y scree plot)

In [ ]:
scaler = StandardScaler()
sensors_scaled = scaler.fit_transform(df_sensors.fillna(0))
pca = PCA(n_components=10, random_state=RANDOM_SEED)
pca.fit(sensors_scaled)
explained = pca.explained_variance_ratio_
cum_explained = np.cumsum(explained)

fig, ax = plt.subplots(figsize=(10,6))
ax.bar(range(1, len(explained)+1), explained, alpha=0.6, label="Individual")
ax.plot(range(1, len(explained)+1), cum_explained, "-o", color="k", label="Cumulative")
ax.set_xlabel("PC")
ax.set_ylabel("Explained variance ratio")
ax.set_title("PCA Scree (first 10 components)")
ax.legend()
save_fig(fig, EDA_OUT / "pca_scree.png")

# Temporal properties — ACF/PACF para 4 sensores

In [ ]:
acf_sensors = ["s2","s3","s7","s15"]
# crear una figura con 4x2 subplots (ACF y PACF por sensor)
fig, axes = plt.subplots(len(acf_sensors), 2, figsize=(14, 3*len(acf_sensors)))
for i, s in enumerate(acf_sensors):
    series = train.groupby("unit")[s].apply(lambda x: x.reset_index(drop=True)).explode().astype(float)  # concatenación simplificada
    u = selected_units[1]  # motor mediano
    series_u = train[train["unit"]==u][s].fillna(method="ffill").values
    ax1 = axes[i,0]
    ax2 = axes[i,1]
    plot_acf(series_u, lags=50, ax=ax1, title=f"ACF {s} (unit {u})")
    plot_pacf(series_u, lags=50, ax=ax2, title=f"PACF {s} (unit {u})")
save_fig(fig, EDA_OUT / "acf_pacf_samples.png")

# RUL trajectories plot

In [ ]:
fig, axes = plt.subplots(3,2, figsize=(14,12))
for ax, u in zip(axes.flatten(), selected_units):
    sub = train[train["unit"]==u]
    ax.plot(sub["cycle"], sub["RUL"], marker="o", linewidth=1)
    ax.set_title(f"Unit {u} — RUL vs cycle")
    ax.set_xlabel("cycle")
    ax.set_ylabel("RUL")
save_fig(fig, EDA_OUT / "rul_trajectories.png")

# Candidate features generation and top_features.csv

In [ ]:
example_sensors = ["s2","s3","s7","s15","s11","s21"]
feats = []
windows = [5,10,20]
for s in example_sensors:
    for w in windows:
        feats.append({
            "name": f"{s}_rm_{w}",
            "description": f"Rolling mean of {s} over window {w}",
            "rationale": "Smooths short-term noise and captures trend"
        })
        feats.append({
            "name": f"{s}_rstd_{w}",
            "description": f"Rolling std of {s} over window {w}",
            "rationale": "Captures local variability and abrupt changes"
        })
    feats.append({
        "name": f"{s}_slope_10",
        "description": f"Slope (OLS) of {s} over window 10",
        "rationale": "Quantifies local increasing/decreasing trend"
    })
# add proximity to max/min per unit
feats.append({"name":"proximity_to_max_s2","description":"(max_s2_unit - s2)/max_s2_unit","rationale":"Relative degradation distance to worst observed value"})
# limit to 12 top features
top_feats = pd.DataFrame(feats)[:12]
top_feats.to_csv(EDA_OUT / "top_features.csv", index=False)

# Leakage checks y recomendaciones 

In [17]:
# Leakage simple check: verificar que RUL no se haya usado para crear features, y que rolling features usan pasado
leakage_issues = []
for c in train.columns:
    if "future" in c.lower():
        leakage_issues.append(c)

print("Leakage checks - suspicious columns:", leakage_issues)
# Documentar manualmente si algo más se detecta. Para rolling: todas las funciones de rolling definidas usan transform(groupby.. rolling) con min_periods and do not peek into future.

Leakage checks - suspicious columns: []


# Exports adicionales y checks automáticos

In [18]:
# Guardar per_unit_stats
per_unit = per_unit_stats(train)
per_unit.to_csv(EDA_OUT / "per_unit_stats.csv", index=False)

# Save additional meta info
meta = {
    "n_rows_train": int(len(train)),
    "n_units_train": int(train["unit"].nunique()),
    "n_rows_test": int(len(test)),
    "n_units_test": int(test["unit"].nunique()),
    "selected_units": selected_units
}
pd.Series(meta).to_frame("value").to_csv(EDA_OUT / "eda_metadata.csv")

# Validaciones automáticas mínimas
checks = {}
checks["files_exist"] = all((EDA_OUT / f).exists() for f in [
    "summary_stats.csv","per_unit_stats.csv","top_features.csv","rul_per_unit.csv",
    "count_units_cycles.png","trajectory_samples.png","correlation_heatmap.png",
    "pca_scree.png","rul_trajectories.png","acf_pacf_samples.png","sensor_vs_op_hexbin.png",
    "summary_stats_table.png"
])
checks["rul_rows_eq_train"] = (len(pd.read_csv(EDA_OUT / "rul_per_unit.csv")) == len(train))
checks["summary_has_missing_pct"] = "missing_pct" in pd.read_csv(EDA_OUT / "summary_stats.csv").columns

print("Checks summary:", checks)

Checks summary: {'files_exist': True, 'rul_rows_eq_train': True, 'summary_has_missing_pct': True}


# Verificación final

In [20]:
print("EDA completo — artifacts guardados en ./eda_outputs/")
print(f"Número de unidades procesadas: {train['unit'].nunique()}")
print("Unidades seleccionadas (6):", selected_units)
print("Top features file:", EDA_OUT / "top_features.csv")
print("Recomendación rápida: usar StandardScaler para sensores; ventanas iniciales prioritarias: 5, 10, 20; priorizar slopes y rolling std como features.")

EDA completo — artifacts guardados en ./eda_outputs/
Número de unidades procesadas: 100
Unidades seleccionadas (6): [39, 79, 69, np.int64(64), np.int64(42), np.int64(97)]
Top features file: c:\Users\jucep\OneDrive\Escritorio\CMAPSS\Mantenimiento-Predictivo-CMAPSS\EDA\eda_outputs\top_features.csv
Recomendación rápida: usar StandardScaler para sensores; ventanas iniciales prioritarias: 5, 10, 20; priorizar slopes y rolling std como features.
